In [1]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [2]:
temp = ConnectDatabase()

In [3]:
temp.conn

<connection object at 0x7f8588ca8398; dsn: 'dbname=postgresdemotwo user=clientdemo password=xxxxxxxxxxx host=postgresdemotwo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [4]:
doc['column_map']['TRANSACTION_MASTER']

{'category': 'category',
 'cust_id': 'id',
 'product_id': 'prod_id',
 'quantity': 'purchasequantity',
 'timestamp': 'date'}

In [5]:
query = 'SELECT %s,%s FROM %s'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                doc['column_map']['TRANSACTION_MASTER']['product_id'],
                                doc['table_map']['TRANSACTION_MASTER'])

In [6]:
print query

SELECT id,prod_id FROM trans


In [7]:
temp.cur.execute(query)

In [8]:
col_names = [desc[0] for desc in temp.cur.description]

In [9]:
col_names

['id', 'prod_id']

In [10]:
import pandas as pd

In [11]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [12]:
trans_table.head()

,id,prod_id
0,3561244241,2507 - 8499 - 24 - OZ
1,3561244241,2507 - 8499 - 24 - OZ
2,3561244241,9753 - 0 - 1 - CT
3,3561244241,9518 - 2988 - 1 - CT
4,3561244241,9518 - 2988 - 1 - CT


In [13]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace=True)

In [14]:
trans_table.head()

,cust_id,product_id
0,3561244241,2507 - 8499 - 24 - OZ
1,3561244241,2507 - 8499 - 24 - OZ
2,3561244241,9753 - 0 - 1 - CT
3,3561244241,9518 - 2988 - 1 - CT
4,3561244241,9518 - 2988 - 1 - CT


In [15]:
trans_table.shape

(71932603, 2)

In [16]:
trans_df = trans_table[['cust_id','product_id']]

In [17]:
grp_cust = trans_df.groupby('cust_id').count()

In [18]:
grp_cust.head()

,product_id
cust_id,
100010021,349
100022923,617
100029473,1644
100050424,1235
100051316,3065


In [19]:
grp_cust = grp_cust.reset_index()

In [20]:
grp_cust.head()

,cust_id,product_id
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [53]:
grp_cust.shape

(62308, 2)

In [21]:
grp_cust.rename(columns={'product_id':'count'}, inplace=True)

In [22]:
grp_cust.head()

,cust_id,count
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [58]:
grp_cust.query('count == 617')

,cust_id,count
1,100022923,617
865,115617935,617
6548,1906583511,617
6850,1932350082,617
7264,1952487136,617
14249,2654463844,617
16159,2803507662,617
16194,2806564621,617
16478,2837827652,617
17809,2959150780,617


In [23]:
grp_cust_df_1 = grp_cust.query('count != 1')

In [24]:
grp_cust_df_1.head()

,cust_id,count
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [25]:
grp_cust_df_1.shape

(62301, 2)

In [26]:
grp_cust.shape

(62308, 2)

In [27]:
grp_cust_df_1.reset_index(drop=True, inplace=True)

In [28]:
grp_cust_df_1.head()

,cust_id,count
0,100010021,349
1,100022923,617
2,100029473,1644
3,100050424,1235
4,100051316,3065


In [29]:
grp_cust_df_1.tail()

,cust_id,count
62296,99804794,529
62297,99818613,673
62298,99844671,1656
62299,99926227,914
62300,99972977,670


In [30]:
cust = grp_cust_df_1[['cust_id']]

In [31]:
cust.head()

,cust_id
0,100010021
1,100022923
2,100029473
3,100050424
4,100051316


In [32]:
final_data = trans_df.merge(cust, on='cust_id')

In [33]:
final_data.shape

(71932596, 2)

In [34]:
cust_prod_comb = final_data.groupby(['cust_id','product_id']).size().reset_index().rename(columns={0:'count'})

In [35]:
cust_prod_comb.head()

,cust_id,product_id,count
0,100010021,1205 - 17292 - 5 - OZ,2
1,100010021,1703 - 10882 - 32 - OZ,1
2,100010021,1703 - 19934 - 26 - OZ,1
3,100010021,1703 - 6608 - 32 - OZ,1
4,100010021,1704 - 27327 - 20 - OZ,1


In [36]:
xyz = cust_prod_comb.merge(grp_cust_df_1, on='cust_id')

In [37]:
xyz.head()

,cust_id,product_id,count_x,count_y
0,100010021,1205 - 17292 - 5 - OZ,2,349
1,100010021,1703 - 10882 - 32 - OZ,1,349
2,100010021,1703 - 19934 - 26 - OZ,1,349
3,100010021,1703 - 6608 - 32 - OZ,1,349
4,100010021,1704 - 27327 - 20 - OZ,1,349


In [38]:
xyz.rename(columns={'count_x':'combination'}, inplace=True)
xyz.rename(columns={'count_y':'visits'}, inplace=True)

In [39]:
xyz.head()

,cust_id,product_id,combination,visits
0,100010021,1205 - 17292 - 5 - OZ,2,349
1,100010021,1703 - 10882 - 32 - OZ,1,349
2,100010021,1703 - 19934 - 26 - OZ,1,349
3,100010021,1703 - 6608 - 32 - OZ,1,349
4,100010021,1704 - 27327 - 20 - OZ,1,349


In [40]:
xyz['affinity'] = xyz['combination'] / xyz['visits'] * 100

In [41]:
xyz.head()

,cust_id,product_id,combination,visits,affinity
0,100010021,1205 - 17292 - 5 - OZ,2,349,0.573066
1,100010021,1703 - 10882 - 32 - OZ,1,349,0.286533
2,100010021,1703 - 19934 - 26 - OZ,1,349,0.286533
3,100010021,1703 - 6608 - 32 - OZ,1,349,0.286533
4,100010021,1704 - 27327 - 20 - OZ,1,349,0.286533


In [42]:
xyz['class'] = ""

In [43]:
xyz.head()

,cust_id,product_id,combination,visits,affinity,class
0,100010021,1205 - 17292 - 5 - OZ,2,349,0.573066,
1,100010021,1703 - 10882 - 32 - OZ,1,349,0.286533,
2,100010021,1703 - 19934 - 26 - OZ,1,349,0.286533,
3,100010021,1703 - 6608 - 32 - OZ,1,349,0.286533,
4,100010021,1704 - 27327 - 20 - OZ,1,349,0.286533,


In [44]:
xyz.loc[xyz['affinity'] >= 50, 'class'] = "Class_A"

In [45]:
xyz.loc[(xyz['affinity'] >= 20) & (xyz['affinity'] < 50) , 'class'] = "Class_B"

In [46]:
xyz.loc[xyz['affinity'] < 20 , 'class'] = "Class_C"

In [49]:
xyz.head(10)

,cust_id,product_id,combination,visits,affinity,class
0,100010021,1205 - 17292 - 5 - OZ,2,349,0.573066,Class_C
1,100010021,1703 - 10882 - 32 - OZ,1,349,0.286533,Class_C
2,100010021,1703 - 19934 - 26 - OZ,1,349,0.286533,Class_C
3,100010021,1703 - 6608 - 32 - OZ,1,349,0.286533,Class_C
4,100010021,1704 - 27327 - 20 - OZ,1,349,0.286533,Class_C
5,100010021,1704 - 9439 - 19 - OZ,1,349,0.286533,Class_C
6,100010021,1708 - 15704 - 24 - OZ,1,349,0.286533,Class_C
7,100010021,1712 - 15197 - 22 - OZ,1,349,0.286533,Class_C
8,100010021,1802 - 2368 - 4 - CT,1,349,0.286533,Class_C
9,100010021,1802 - 51733 - 1 - CT,1,349,0.286533,Class_C


In [52]:
xyz.query('affinity >= 50')

,cust_id,product_id,combination,visits,affinity,class
4562039,2318817479,6408 - 13918 - 5 - LT,194,379,51.187335,Class_A
4922048,2415882960,9781 - 8255 - 1 - CT,101,183,55.191257,Class_A
6063491,2595430565,6408 - 1959 - 0.75 - LT,1,2,50.000000,Class_A
6063492,2595430565,6408 - 38185 - 1.5 - LT,1,2,50.000000,Class_A
6190416,2620462049,9753 - 0 - 1 - CT,119,218,54.587156,Class_A
8275726,3020234434,9732 - 10312 - 1 - CT,1,2,50.000000,Class_A
8275727,3020234434,9753 - 0 - 1 - CT,1,2,50.000000,Class_A
8283514,3020705749,2035 - 3611 - 64 - OZ,8,12,66.666667,Class_A
9463902,3356477529,9753 - 0 - 1 - CT,81,150,54.000000,Class_A
9919671,3413740410,9753 - 0 - 1 - CT,141,282,50.000000,Class_A
